In [2]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np

In [18]:
def daily_temp_update(filename, station):
    all_sst = pd.read_csv(filename)
    updated_year = all_sst.YEAR.iloc[-1]
    updated_month = all_sst.MONTH.iloc[-1]
    updated_day = all_sst.DAY.iloc[-1]
    updated_date = datetime(updated_year,updated_month,updated_day)

    if updated_day == 1:
        suffix = 'st'
    elif updated_day == 2:
        suffix = 'nd'
    elif updated_day == 3:
        suffix = 'rd'
    elif (4 <= updated_day <= 20):
        suffix = 'th'
    elif updated_day == 21:
        suffix = 'st'
    elif updated_day == 22:
        suffix = 'nd'
    elif updated_day == 23:
        suffix = 'rd'
    elif (24 <= updated_day <= 30):
        suffix = 'th'
    elif updated_day == 31:
        suffix = 'st'

    climatology = all_sst[(all_sst['MONTH'] == updated_month) & (all_sst['DAY'] == updated_day)]

    today_sst = 19 #all_sst['SURF_TEMP_C'].iloc[-1]
    today_sst_F = round((today_sst*(9/5))+32,2)

    trivia_msg = ''
    if today_sst > np.nanpercentile(climatology.SURF_TEMP_C.values,90): #How do you define a MHW day? Also np.percentile assumes a normal distribution I think
        trivia_msg = 'Today is among \nthe hottest 10% of \n' + updated_date.strftime('%B %-d') + suffix + 's on record'
    if today_sst >= climatology.SURF_TEMP_C.max():
        trivia_msg = 'Today is the hottest \n' + updated_date.strftime('%B %-d') + suffix + ' on record'
    elif today_sst <= climatology.SURF_TEMP_C.min():
        trivia_msg = 'Today is the coldest \n' + updated_date.strftime('%B %-d') + suffix + ' on record'
    last_updated_date = datetime(all_sst.YEAR.iloc[-1], all_sst.MONTH.iloc[-1], all_sst.DAY.iloc[-1]).strftime('%B %d, %Y')
    if len(str(all_sst.TIME_PST.iloc[-1])) == 3:
        last_updated_time = '0'+str(all_sst.TIME_PST.iloc[-1])[0]+':'+str(all_sst.TIME_PST.iloc[-1])[1:]
    else:
        last_updated_time = str(all_sst.TIME_PST.iloc[-1])[0:2]+':'+str(all_sst.TIME_PST.iloc[-1])[2:]

    fig, ax = plt.subplots(figsize=(18, 12), dpi=500)
    
    cm = plt.colormaps.get_cmap('RdBu_r')
    n, bins, patches = ax.hist(climatology.SURF_TEMP_C, bins=15)
    ax.axvline(today_sst, color='k', linestyle='dashed', linewidth=1)
    ax.set_xlabel('Temperature (˚C)', fontsize=30)
    ax.legend(["Today's Temperature"], fontsize=30)


    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    col = bin_centers - min(bin_centers)
    col /= max(col)

    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))

    ax.tick_params(axis='x', labelsize=30)
    ax.yaxis.set_ticks([])

    fahrenheit_bins = [(b * 9/5) + 32 for b in bins]
    fahrenheit_labels = [str(round(b)) for b in fahrenheit_bins]

    x = np.arange(0, 10, 0.1)
    y = 1

    plt.annotate("{}˚C".format(today_sst), fontsize=120, fontweight='bold', c='steelblue', xy=(1.05, 0.85), xycoords="axes fraction", ha='left')
    plt.annotate('  ' + str(today_sst_F) + '˚F', fontsize=80, fontweight='bold', c='skyblue', xy=(1.05, 0.72), xycoords="axes fraction", ha='left')
    plt.annotate("Measured on:", fontsize=30, style='italic', c='grey', xy=(1.05, 0.62), xycoords="axes fraction", ha='left')
    plt.annotate(last_updated_date, fontsize=45, weight='bold', c='black', xy=(1.05, 0.55), xycoords="axes fraction", ha='left')
    plt.annotate("at " + last_updated_time, fontsize=35, weight='bold', c='dimgrey', xy=(1.05, 0.49), xycoords="axes fraction", ha='left')
    if trivia_msg != '':
        plt.annotate(trivia_msg, fontsize=35, c='dimgrey', xy=(1.05, 0), xycoords="axes fraction", ha='left')
    plt.show()
    #plt.savefig('/Volumes/sccoos/manual_shore_station/DATA/TEMPERATURE/AutomatedPlotsForMSSwebsite/SIO_surf_temp_now.png', bbox_inches='tight', dpi=500)


In [19]:
filename='SIO_TEMP_20230105.csv'
daily_temp_update(filename, 'sio')

: 